# How to work with pp_network.py

*pp_network.py* has a class called *net_model* that handles the whole network side of our simulation. The network state object includes a pandapower object called *net* as well as methods to implement actions, calculate the reward, and run simulations on *net*.

Another matpower note: net.line has an attribute 'in_service' so it should be easy for us to simulate a fallen line


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandapower as pp
import pandapower.networks

In [2]:
import pp_network
from pp_network import *

# Trouble Shoot 

Try to figure out the rural_1 network, for example

In [16]:
network_name = 'rural_1'
abc = pp.networks.create_synthetic_voltage_control_lv_network(network_class = network_name)
# abc(network_name)

In [18]:
abc

This pandapower network includes the following parameter tables:
   - bus (26 elements)
   - load (14 elements)
   - sgen (5 elements)
   - ext_grid (1 element)
   - line (24 elements)
   - trafo (1 element)
   - bus_geodata (26 elements)

In [12]:
pp.networks.create_synthetic_voltage_control_lv_network(network_class = network_name)

This pandapower network includes the following parameter tables:
   - bus (26 elements)
   - load (14 elements)
   - sgen (5 elements)
   - ext_grid (1 element)
   - line (24 elements)
   - trafo (1 element)
   - bus_geodata (26 elements)

In [3]:
network_model = pp_network.net_model('rural_1')

AttributeError: 'function' object has no attribute 'rural_1'

## Initialize network

Options to initialize: 'case5','case9','case14','rural_1','rural_2','village_1','village_2','suburb_1','iceland',etc... Basically pick anything from https://pandapower.readthedocs.io/en/v1.6.0/networks.html

In [3]:
network_model = pp_network.net_model('case14')

In [4]:
print('Number of load buses: ',network_model.num_loadbus)
print('Which buses are they on: ',network_model.loadbuses)
print('Number of generator buses: ',network_model.num_sgen)
print('Number of normal generators: ',network_model.num_normalgen)

Number of load buses:  11
Which buses are they on:  [ 1  2 13  3  4  5  8  9 10 11 12]
Number of generator buses:  0
Number of normal generators:  4


### Fake test data

In [5]:
num_times = 24*60
p_load_data = 1.0*np.ones((network_model.num_loadbus,num_times))
q_load_data = 0.0*np.ones((network_model.num_loadbus,num_times))

## Add generators

In [6]:
network_model.add_generation(7,1.0) 
# Add a generator at bus 7 and initialize its real power production as 1 kW
network_model.add_generation(10,1.0)

# Add battery

In [7]:
network_model.add_battery(3,1.0,10,0.0) # Initialize at bus 3 with power flow 1.0 (charging), capacity 10 kWh, initialy SOC 0 %

## Update loads and run a powerflow

In [8]:
network_model.update_loads(p_load_data[:,0],q_load_data[:,0])

In [9]:
network_model.run_powerflow(False, True)

In [10]:
network_model.energy_capacities

array([10.])

# Inspect results

In [12]:
# print('Real powerflows on the lines: ')
# print(network_model.p_lineflow)

# Key Documentation: 

https://pandapower.readthedocs.io/en/v1.6.0/elements.html

Continuing to work on this and will check it is working